In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
#%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
import lightgbm
from lightgbm import LGBMClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import warnings
import argparse
warnings.filterwarnings('ignore')
'''
parser = argparse.ArgumentParser()
parser.add_argument('--alpha')
parser.add_argument('--min_periods')
args = parser.parse_args()

alpha=float(args.alpha)
min_periods = int(args.min_periods)
'''
alpha = 0.9
min_periods=1
def sigma(alpha, period):
    r = 1-alpha
    return (r**(period)-1)/(r-1)

file = './data/Retrosheet_2010_2019/2010_to_2019_seasons.csv'
game_df = pd.read_csv(file)

game_col_del = '''VisitorGDP,VisitorCI,HomeGDP,HomeCI, DoubleHeader, DayOfWeek, VisitingTeamLeague, HomeTeamLeague, DayNight, CompletionInfo,ForfeitInfo, ProtestInfo,Attendence, Duration, VisitorLineScore, HomeLineScore, UmpireHID, UmpireHName, Umpire1BID, Umpire1BName, Umpire2BID,Umpire2BName, Umpire3BID, Umpire3BName, UmpireLFID, UmpireLFName, UmpireRFID,UmpireRFName, VisitorManagerID, VisitorManagerName, HomeManagerID, HomeManagerName,WinningPitcherID, WinningPitcherName, LosingPitcherID, LosingPitcherNAme,SavingPitcherID, SavingPitcherName, GameWinningRBIID, GameWinningRBIName, VisitorBatting1Position, VisitorBatting2Position, VisitorBatting3Position,VisitorBatting4Position, VisitorBatting5Position, VisitorBatting6Position,VisitorBatting7Position, VisitorBatting8Position, VisitorBatting9Position,HomeBatting1Position, HomeBatting2Position, HomeBatting3Position,HomeBatting4Position, HomeBatting5Position, HomeBatting6Position,HomeBatting7Position, HomeBatting8Position, HomeBatting9Position,AdditionalInfo, AcquisitionInfo'''
game_col_del = game_col_del.replace(" ", "")
game_col_del = game_col_del.split(",")

game_df.drop(game_col_del,axis=1,inplace=True)


col_del = ['VisitorBatting1PlayerID', 'VisitorBatting1Name', 'VisitorBatting2PlayerID', 'VisitorBatting2Name', 'VisitorBatting3PlayerID', 'VisitorBatting3Name', 'VisitorBatting4PlayerID', 'VisitorBatting4Name', 'VisitorBatting5PlayerID', 'VisitorBatting5Name', 'VisitorBatting6PlayerID', 'VisitorBatting6Name', 'VisitorBatting7PlayerID', 'VisitorBatting7Name', 'VisitorBatting8PlayerID', 'VisitorBatting8Name', 'VisitorBatting9PlayerID', 'VisitorBatting9Name', 'HomeBatting1PlayerID', 'HomeBatting1Name', 'HomeBatting2PlayerID', 'HomeBatting2Name', 'HomeBatting3PlayerID', 'HomeBatting3Name', 'HomeBatting4PlayerID', 'HomeBatting4Name', 'HomeBatting5PlayerID', 'HomeBatting5Name', 'HomeBatting6PlayerID', 'HomeBatting6Name', 'HomeBatting7PlayerID', 'HomeBatting7Name', 'HomeBatting8PlayerID', 'HomeBatting8Name', 'HomeBatting9PlayerID', 'HomeBatting9Name']
game_df.drop(col_del,axis=1,inplace=True)
game_df = game_df.rename(columns={"VisitingTeam":"VisitorTeam","VisitingTeamGameNumber":"VisitorTeamGameNumber"})


away_inning = game_df['LengthInOuts']//2
home_inning = game_df['LengthInOuts']- away_inning


game_df['VisitorOffInn'] = away_inning
game_df['VisitorDifInn'] = home_inning
game_df['HomeOffInn'] = home_inning
game_df['HomeDifInn'] = away_inning

game_df['Date'] = pd.to_datetime(game_df['Date'].astype(str), format='%Y%m%d')
game_df['current_year'] = game_df['Date'].dt.year
game_df['Home_team_won?'] = game_df['HomeRunsScore'] > game_df['VisitorRunsScored']
game_df['Visitor_team_won?'] = game_df['HomeRunsScore'] < game_df['VisitorRunsScored']


bat_stat = ['TeamGameNumber','Team','AB','H','D','T','HR','RBI','SH','SF',
'HBP','BB','IBB','K','SB','CS','LOB','Pitchers',
'ER','TER','WP','Balks','PO','A','E','Passed','DB','TP','OffInn','DifInn','_team_won?']
pit_stat = ['H', 'D', 'T', 'HR', 'RBI', 'SH', 'SF', 'HBP', 'BB', 'IBB', 'K', 'LOB', 'ER','DB','AB']


select_stat = ['Date']
select_stat += ['Home'+i for i in bat_stat]+['Visitor'+i for i in pit_stat]
rename1 = {'Visitor'+i:'Pit'+i for i in pit_stat}
rename2 = {'Home'+i:i for i in bat_stat}


## Home 시각 
select_stat = ['Date']
select_stat += ['Home'+i for i in bat_stat]+['Visitor'+i for i in pit_stat]
rename1 = {'Visitor'+i:'Pit'+i for i in pit_stat}
rename2 = {'Home'+i:i for i in bat_stat}
Home = game_df.copy()
Home = Home[select_stat].rename(columns=rename1)
Home = Home.rename(columns=rename2)
# away 시각
select_stat = ['Date']
select_stat += ['Visitor'+i for i in bat_stat]+['Home'+i for i in pit_stat]
rename1 = {'Home'+i:'Pit'+i for i in pit_stat}
rename2 = {'Visitor'+i:i for i in bat_stat}
Visitor = game_df.copy()
Visitor = Visitor[select_stat].rename(columns=rename1)
Visitor = Visitor.rename(columns=rename2)

sep_team = pd.concat([Home,Visitor])
sep_team = sep_team.sort_values(['Date','TeamGameNumber'],ascending=True)
sep_team = sep_team.reset_index(drop=True)
sep_team = sep_team.rename(columns={'PitDB':'DP'})
col = ['AB', 'H', 'D', 'T', 'HR', 'RBI','DP',
       'SF', 'BB', 'K', 'SB', 'CS', 'LOB', 'Pitchers', 'ER', 'TER', 'WP','PitAB',
       'Balks', 'PO', 'A', 'E', 'Passed', 'DB', 'TP', 'OffInn', 'DifInn',
       'PitH', 'PitD', 'PitT', 'PitHR', 'PitRBI', 'PitSF',
       'PitBB', 'PitK', 'PitLOB', 'PitER','SH','HBP','PitSH','PitHBP','PitIBB']
a = sep_team.copy()
sep_team['year']= sep_team.Date.dt.year

In [44]:
s = pd.DataFrame([])
for team in sep_team.Team.unique():
    for year in sep_team.year.unique():
        temp = sep_team[(sep_team['Team']==team)&(sep_team.year==year)].copy()
        #temp.TeamGameNumber+=1
        temp = temp.sort_values(by=['TeamGameNumber'])
        temp = temp.reset_index(drop=True)
        #temp = temp.set_index('Date')
        temp[col] = temp[col].ewm(alpha=alpha,min_periods=min_periods).mean()
        date = list(temp.Date)
        date = date[1:]
        date.append(0)
        #date.insert(0,0)
        #date.pop()
        temp.Date  =date

        s = pd.concat([s,temp])

In [45]:
temp

,Date,TeamGameNumber,Team,AB,H,D,T,HR,RBI,SH,...,PitSF,PitHBP,PitBB,PitIBB,PitK,PitLOB,PitER,DP,PitAB,year
0,2019-03-29 00:00:00,1,MIA,32.000000,4.000000,0.000000,0.000000,3.000000,3.000000,0.000000,...,0.000000,0.000000,3.000000,0.000000,7.000000,6.000000,3.000000,0.000000,36.000000,2019
1,2019-03-30 00:00:00,2,MIA,27.454545,2.181818,0.000000,0.000000,1.181818,1.181818,0.000000,...,0.000000,0.000000,3.909091,0.000000,5.181818,8.727273,1.181818,0.909091,37.818182,2019
2,2019-03-31 00:00:00,3,MIA,36.954955,14.630631,3.603604,0.000000,0.117117,6.423423,0.000000,...,0.900901,0.900901,0.387387,0.000000,13.126126,6.270270,6.423423,0.990991,34.378378,2019
3,2019-04-01 00:00:00,4,MIA,28.894689,5.962196,0.360036,0.000000,0.911791,3.342034,0.000000,...,0.090009,0.090009,0.938794,0.000000,7.612061,5.126913,3.342034,0.999100,30.437444,2019
4,2019-04-02 00:00:00,5,MIA,34.389524,8.696247,0.036000,0.000000,0.991180,3.034200,0.000000,...,0.009000,1.809018,1.893889,0.000000,15.161282,5.912699,1.234182,1.899919,35.443794,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,2019-09-26 00:00:00,158,MIA,32.132782,4.713131,1.182090,0.001801,1.008911,3.136892,0.009000,...,1.800000,0.990000,4.020869,0.090990,9.878459,7.308124,0.436891,0.918928,34.647180,2019
158,2019-09-27 00:00:00,159,MIA,32.913278,5.871313,2.818209,0.000180,2.800891,3.913689,0.000900,...,1.080000,0.999000,4.002087,0.009099,11.787846,6.130812,3.643689,0.091893,29.564718,2019
159,2019-09-28 00:00:00,160,MIA,54.591328,15.887131,1.181821,0.000018,1.180089,3.091369,1.800090,...,1.908000,1.899900,5.800209,1.800910,13.778785,13.213081,3.964369,0.909189,47.956472,2019
160,2019-09-29 00:00:00,161,MIA,35.159133,7.888713,1.018182,0.900002,1.018009,3.009137,0.180009,...,0.190800,1.089990,5.080021,0.180091,4.077878,6.721308,3.096437,0.990919,34.495647,2019


In [46]:
sep_team[(sep_team['Team']==team)&(sep_team.year==year)]

,Date,TeamGameNumber,Team,AB,H,D,T,HR,RBI,SH,...,PitSF,PitHBP,PitBB,PitIBB,PitK,PitLOB,PitER,DP,PitAB,year
43742,2019-03-28,1,MIA,32,4,0,0,3,3,0,...,0,0,3,0,7,6,3,0,36,2019
43771,2019-03-29,2,MIA,27,2,0,0,1,1,0,...,0,0,4,0,5,9,1,1,38,2019
43799,2019-03-30,3,MIA,38,16,4,0,0,7,0,...,1,1,0,0,14,6,7,1,34,2019
43829,2019-03-31,4,MIA,28,5,0,0,1,3,0,...,0,0,1,0,7,5,3,1,30,2019
43855,2019-04-01,5,MIA,35,9,0,0,1,3,0,...,0,2,2,0,16,6,1,2,36,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48464,2019-09-25,158,MIA,31,4,1,0,1,3,0,...,2,1,4,0,10,7,0,1,34,2019
48494,2019-09-26,159,MIA,33,6,3,0,3,4,0,...,1,1,4,0,12,6,4,0,29,2019
48521,2019-09-27,160,MIA,57,17,1,0,1,3,2,...,2,2,6,2,14,14,4,1,50,2019
48553,2019-09-28,161,MIA,33,7,1,1,1,3,0,...,0,1,5,0,3,6,3,1,33,2019


In [48]:

#sep_team[col] = sep_team.groupby(['year','Team'])[col].transform(lambda x: x.ewm(alpha=alpha,min_periods=min_periods,adjust=True).mean())*sigma(alpha,min_periods)
sep_team[col] = sep_team.groupby(['year','Team'])[col].transform(lambda x: x.ewm(alpha=alpha,min_periods=min_periods,adjust=True).sum())
#sep_team[col] = sep_team[col].subtract(a[col])
#sep_team.TeamGameNumber -= 1

#sep_team[col] = a[col]
sep_team = sep_team[sep_team['TeamGameNumber']>=10]
#sep_team['TeamGameNumber'] += 10
select_feature = ['Date', 'TeamGameNumber', 'Team','_team_won?']
# feature selection
select_feature += ['AB', 'H', 'D', 'T', 'HR', 'RBI','BB', 'K','DP', 'HBP','IBB','SF','SH',#bat stat
                    'ER','OffInn','DifInn' # game info
                    ,'PitAB','PitH', 'PitD', 'PitT', 'PitHR', 'PitRBI', 'PitSF', 'PitBB',"PitIBB","PitHBP" ,'PitK','PitLOB', 'PitER'
                    ]
sep = sep_team[select_feature].copy()
# H, OBP, SLG, ISO, Babip, RC, wOBA X, K, BB, RBI, K/B, HR   
# 타자 가공
sep['RC'] = ((sep['H']+sep['BB']+sep['HBP']-sep['DP'])
        *(sep['H']+2*sep['D']+3*sep['T']+4*sep['HR']+0.52*(sep['SF']+sep['SH'])+0.26*(sep['BB']+sep['HBP']-sep['IBB']))
        )/(sep['AB']*sep['AB']) #득점 생산


#사구,사사구, 고의사구 통합
sep['BB'] += sep['HBP']+sep['IBB']
sep['PitBB'] += sep['PitHBP']+sep['PitIBB']
#del sep['HBP'],sep['IBB'],sep['PitHBP'],sep['PitIBB']
# 희생번트, 희생플라이 통합
sep['PitSF'] += sep_team['PitSH']
sep['SF'] += sep['SH']
#del sep['SH'],sep['PitSH']

sep['K/B'] = sep['K']/(sep['BB']+0.5)
sep['PA'] = sep['AB'] - sep['BB'] - sep['SF'] # 타수
sep['Babip'] = (sep['H']+sep['D']+sep['T']) / (sep['PA']-sep['K']-sep['HR']-sep['SF']) # 인플레이 타구
sep['SLG'] = (sep['H']+2*sep['D']+3*sep['T']+4*sep['HR'])/sep['PA'] # 장타율
sep['H'] = sep['H']/sep['PA'] # 안타율
sep['D'] = sep['D']/sep['PA'] # 안타율
sep['T'] = sep['T']/sep['PA'] # 안타율
sep['HR'] = sep['HR']/sep['PA'] # 안타율
sep['BB'] = sep['BB']/sep['PA'] # 볼넷
sep['K'] = sep['K']/sep['PA']
sep['RBI'] = sep['RBI']/sep['TeamGameNumber'] # 득점

sep['OBP'] = (sep['H']+sep['BB']) #출루율
sep['OPS'] = sep['SLG']+sep['OBP'] #OPS
sep['GPA'] = (1.8*sep['OBP']+sep['SLG'])/4#GPA(Gross Production Average) - park factor를 적용해야함
sep['ISO'] = sep['SLG'] - sep['H']

# 투수 가공
# PitH, PitOBP, PitSLG, PitOPS, PitHR, EAR, PitB, PitK, PitK/B, WHIP, kwERA
sep['ERA'] = sep['PitER']/sep['DifInn']
sep['TotalH'] = sep['PitH']+sep['PitD']+sep['PitT']+sep['PitHR']
sep['TotalBB'] = sep['PitBB']+sep['PitHBP']
sep['PitPA'] = sep['PitAB'] - sep['TotalBB'] - sep['PitSF']
sep['kwERA'] = (5.40-12*(sep['PitK'] - sep['TotalBB']))/sep['PitPA']
sep['WHIP'] = (sep['TotalH']+sep['TotalBB'])/sep['DifInn']
sep['PitK'] = sep['PitK']/sep['DifInn']
sep['PitB'] = sep['TotalBB']/sep['DifInn']
sep['PitK/B'] = sep['PitK']/(sep['PitB']+0.5)


sep['PitSLG'] = (sep['PitH']+2*sep['PitD']+3*sep['PitT']+4*sep['PitHR'])/sep['PitPA'] # 피장타율
sep['PitH'] = sep['PitH']/sep['PitPA'] # 피안타율
sep['PitOBP'] = (sep['TotalH']+sep['TotalBB']) / sep['PitAB'] #피출루율
sep['PitOPS'] = sep['PitSLG']+sep['PitOBP'] #피OPS
sep['PitHR'] /= sep['PitPA']

#temp = sep[['Date', 'TeamGameNumber', 'Team', '_team_won?','GPA', 'OPS','OBP','H','SLG','Babip','RC']]
temp = sep
game_log = game_df[['Date','VisitorTeam','HomeTeam','VisitorTeamGameNumber','HomeTeamGameNumber']]
#game_log.VisitorTeamGameNumber-=1
#game_log.HomeTeamGameNumber -=1
game_log = game_log[game_log.VisitorTeamGameNumber>=5]
game_log = game_log[game_log.HomeTeamGameNumber>=5]
temp['year'] = temp.Date.dt.year
game_log['year'] = game_log.Date.dt.year

In [26]:
#temp.TeamGameNumber+=3

In [49]:
game_log = pd.merge(game_log,temp,left_on = ['Date','VisitorTeam'], right_on = ['Date','Team'],how='left')
game_log = pd.merge(game_log,temp,left_on = ['Date','HomeTeam'], right_on = ['Date','Team'],how='left')


game_log['year'] = game_log.Date.dt.year
train = game_log[game_log.year!=2019]
test = game_log[game_log.year==2019]

# 'H','OBP','SLG','ISO','Babip','RC','K','BB','RBI','K/B','HR','PitH','PitOBP','PitSLG','PitOPS','PitHR','EAR','PitB','PitK','PitK/B','WHIP'

y_cols = ['_team_won?_x']
select = ['H','OBP','SLG','ISO','Babip','RC','K','BB','RBI','K/B','HR','PitH','PitOBP','PitSLG','PitOPS','PitHR','ERA','PitB','PitK','PitK/B','WHIP']
#select = ['PitH','PitOBP','PitSLG','PitOPS','PitHR','ERA','PitB','PitK','PitK/B','WHIP']

#select = ['H', 'D', 'T', 'HR', 'RBI', 'BB']
x_cols = [i+'_x' for i in select]
x_cols += [i+'_y' for i in select]
train = train.dropna()
test = test.dropna()

y_train = train[y_cols]*1
y_test = test[y_cols]*1

X_train = train[x_cols]
X_test = test[x_cols]
x_ = [i+'_x' for i in select]
y_ = [i+'_y' for i in select]

In [ ]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression()


#results = sm.Logit(y_train,X_train).fit()
#print(results.summary())
results = model.fit(X_train,y_train.astype('int').to_numpy().flatten())


y_hat = results.predict( X_train)
y_hat = list(map(round,y_hat))
print('Train accuracy = ', accuracy_score(y_train.astype('int').to_numpy().flatten(), y_hat))

y_hat = results.predict( X_test)
y_hat = list(map(round,y_hat))
print('Test accuracy = ', accuracy_score(y_test.astype('int').to_numpy().flatten(), y_hat))
